# 0. 모듈 & 라이브러리

In [1355]:
import pandas as pd  
import numpy as np   

import requests
from bs4 import BeautifulSoup    
from selenium import webdriver   
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller 

import time    
import re
import math

import warnings
warnings.filterwarnings('ignore')

In [1356]:
pwd

'c:\\Users\\Jay\\Desktop\\python_ML'

In [1]:
df = pd.read_excel('gyobo_best_all.xlsx') 
df = df[df['담당자']==3]
df

NameError: name 'pd' is not defined

In [1358]:
url = df.iloc[44][1]
url

9791160361339

In [1359]:
title = df['도서명'][44]
title

'매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)'

# 1. 데이터 크롤링

In [1360]:
# 크롬 웹 브라우저 실행 (keyword 검색결과)

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.get('http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&linkClass=01&barcode={}'.format(url)) # query : 바코드 숫자

In [1361]:
# 회원리뷰 클릭 : 그래야 clova 회원리뷰 목록이 나옴
driver.find_element_by_partial_link_text('회원리뷰').click() 
time.sleep(2)

In [1362]:
## 전체 리뷰 수 전처리 함수 : 정규표현식 이용해서 숫자만 남기고 int 변환
def preprocessing(total):
    k_total = int(re.sub('\D','',total))
    return k_total

In [1363]:
## 전처리 함수 완성 ## 

keyword_list = []
id_list = []
klover_list = []
comment_list = []

def keyword_pre(keywords):
    for keyword in keywords:
        keyword_list.append(keyword.text)

def id_pre(ids):
    for id in ids:
        id_list.append(id.text)
        
def klovers_pre(klovers):
    for klover in klovers:
        klover_list.append(klover.text)
        
def comments_pre(comments):
    for comment in comments:
        comment_list.append(comment.text)

In [1364]:
# 알고리즘 모수 구하기

In [1365]:
total = driver.find_element_by_id('kloverTotal').text
k_total = preprocessing(total)
k_total

5

In [1366]:
page_n = math.ceil(k_total / 5)
page_n

1

In [1367]:
n =page_n -page_n % 10
n

0

In [1368]:
m= page_n %10+3
m

4

In [1369]:
# 키워드 수집
keywords = driver.find_elements_by_class_name('book_keyword > a')
keyword_pre(keywords)

# 스코어 수집 
score = driver.find_element_by_class_name('score').text
score

i = 0

while i < page_n :
    ids = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .id')
    id_pre(ids)
   
    klovers = driver.find_elements_by_css_selector('.comment_wrap .kloverRating > span')
    klovers_pre(klovers)

    comments = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .comment')
    comments_pre(comments)
    
    
    i +=1
    time.sleep(1) 
    
    if i> n or k_total <= 50:
        index = m
    else:
        index = 13 
    
    if i < page_n:
        
        css_selector= '.box_detail_review > div.list_paging.align_center > div > a:nth-child({})'.format(index)
        driver.find_element_by_css_selector(css_selector).click()
    else:
        print('Done!!')
        driver.close()
    
    time.sleep(2)
    



Done!!


In [1370]:
keyword_list

['디저트', '유기농', '메이커', '마테오', '쉐프', '포스팅']

In [1371]:
len(id_list), len(klover_list), len(comment_list)

(5, 5, 5)

# 2. dataframe 만들기

In [1372]:
df

,순위,바코드,도서명,저자명,출판사명,출간일,정가,카테고리,담당자
0,101,9791190641159,첫 번째 디자인 케이크,지은혜(아이라이크케이크),책밥,20200820,16000,요리,1
1,79,9788982814471,연금술사,파울로 코엘료,문학동네,20181205,12000,소설,1
2,138,9791189727451,기계설비 표준품셈(2022)(CD1장포함)(양장본 HardCover),건설연구원,건설연구원,20220114,34000,기술/공학,1
3,115,9791167900791,그때 그 마음(2022 제67회 현대문학상 수상소설집),정소현 외,현대문학,20211210,15000,소설,1
4,89,9791165343842,대한민국 금기 깨기,김동연,쌤앤파커스,20210728,17000,정치사회,1
...,...,...,...,...,...,...,...,...,...
915,52,3904000300085,뉴필로소퍼 (2022년 Vol. 17),뉴필로소퍼 편집부,바다출판사,20220106,15000,잡지,1
916,43,7051769003744,요시타케 신스케 작가 시리즈 전8권 세트(창작동화2권 증정) : 신간 심심해 심심해 포함,요시타케 신스케,주니어김영사/스콜라/봄나무,20170830,91000,어린이전집,1
917,326,9780201825930,Exploring English 1.(Work Book),"Harris, Tim",Prentice-Hall,19950901,9500,어린이이엘티,1
918,65,9788998441074,나는 소망한다 내게 금지된 것을,양귀자,쓰다,20190420,15000,소설,1


In [1373]:
data = [(id_list,klover_list,comment_list)]
lable = ['id','klover','comment']
df44= pd.DataFrame(zip(id_list,klover_list,comment_list),columns=lable)

In [1374]:
title

'매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)'

In [1375]:
# 키워드, 스코어, 리뷰 수 컬럼 추가
keyword_col = ' '.join(keyword_list)
df44['keyword'] = keyword_col
df44['score']= score
df44['total'] = k_total
df44['title'] = title


In [1376]:
df44

,id,klover,comment,keyword,score,total,title
0,vi**3025,4,f매거진은 해당 주제에 관해 넓고 깊게 잘 알려주어 너무 유용합니다. 물론 재미도 있고요,디저트 유기농 메이커 마테오 쉐프 포스팅,10.0,5,매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)
1,pr**igy051,4,내용도 알차고 볼거리 도 많아서 좋습니다,디저트 유기농 메이커 마테오 쉐프 포스팅,10.0,5,매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)
2,ky**412,4,쉽고 재미있게 잘 읽혀요 유익합니다,디저트 유기농 메이커 마테오 쉐프 포스팅,10.0,5,매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)
3,ru**ia12,4,읽을만한 좋은 책입니다.,디저트 유기농 메이커 마테오 쉐프 포스팅,10.0,5,매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)
4,vo**ejun,4,좋아하는 매건진이에요.,디저트 유기농 메이커 마테오 쉐프 포스팅,10.0,5,매거진 F(Magazine F) No. 17: 아이스크림(ICE CREAM)(한글판)


# 3. excel로 저장

In [1377]:
# 도서명 그대로 저장
df44.to_excel('{}.xlsx'.format(title))